# Mixed Integer Programming(MIP) for the MnM Project

In [1]:
import glob, os
import numpy as np
import pandas as pd
from pyomo.environ import *
os.chdir('/Users/yul/Desktop/Github Fetch/stochastic_programming/5. MnM Project')

In [2]:
nutrition = pd.read_csv('nutrition.csv')
nutrition = nutrition.sort_values(by='title').reset_index()
nutrition = nutrition.drop(['index'],axis=1)
nutrition.head(5)

,Unnamed: 0,title,calories,fat,protein,sodium
0,113,Baked Feta and Greens with Lemony Yogurt,577.0,38.0,23.0,839.0
1,49,"Butternut Squash, Kale, and Crunchy Pepitas Taco",233.0,17.0,7.0,363.0
2,77,"Curried Lentil, Tomato, and Coconut Soup",437.0,28.0,13.0,667.0
3,104,Curried Pumpkin Soup,165.0,14.0,2.0,373.0
4,29,Easy General Tso's Chicken,699.0,35.0,50.0,1189.0


In [3]:
svd = pd.read_csv('full_afterSVD.csv')
svd = svd.tail(1)
svd = pd.DataFrame(svd.T[2:])
svd = svd.reset_index()
svd = svd.sort_values(by='index').reset_index()
svd.head(5)

,level_0,index,53
0,2,Baked.Feta.and.Greens.with.Lemony.Yogurt,1.000000
1,18,Butternut.Squash..Kale..and.Crunchy.Pepitas.Taco.,3.000000
2,9,Curried.Lentil..Tomato..and.Coconut.Soup.,3.695717
3,10,Curried.Pumpkin.Soup,2.000000
4,4,Easy.General.Tso.s.Chicken,3.398837


In [4]:
nutrition['rating'] = svd[53]

In [5]:
nutrition.head(5)

,Unnamed: 0,title,calories,fat,protein,sodium,rating
0,113,Baked Feta and Greens with Lemony Yogurt,577.0,38.0,23.0,839.0,1.000000
1,49,"Butternut Squash, Kale, and Crunchy Pepitas Taco",233.0,17.0,7.0,363.0,3.000000
2,77,"Curried Lentil, Tomato, and Coconut Soup",437.0,28.0,13.0,667.0,3.695717
3,104,Curried Pumpkin Soup,165.0,14.0,2.0,373.0,2.000000
4,29,Easy General Tso's Chicken,699.0,35.0,50.0,1189.0,3.398837


In [6]:
nutrition = nutrition.sort_values(by='Unnamed: 0')
nutrition.head(5)

,Unnamed: 0,title,calories,fat,protein,sodium,rating
5,2,"Easy Green Curry with Chicken, Bell Pepper, an...",549.0,36.0,42.0,760.0,3.814655
11,10,"Persian-Spiced Chicken with Spaghetti Squash, ...",704.0,44.0,49.0,1002.0,5.000000
6,13,Istanbul-Style Wet Burger (Islak Burger),691.0,34.0,46.0,779.0,4.000000
13,15,Pumpkin Spice Bundt Cake with Buttermilk Icing,372.0,14.0,5.0,282.0,1.000000
12,18,Pumpkin Cheesecake with Bourbon–Sour Cream Top...,429.0,32.0,6.0,316.0,1.641707


In [7]:
# Define parameters
# recipe matrix 
rating = nutrition['rating'].tolist()
calories = nutrition['calories'].tolist()
protein = nutrition['protein'].tolist()
fat = nutrition['fat'].tolist()
sodium = nutrition['sodium'].tolist()

# We have determined :
price = [7.2, 5.1,4.3,3.1,4.5,4.3,5.2,9.8,6.1,6.5,4.5,5.5,3.5,13,12.5,4.8,7.5,3.5,6.3,5]
p_times = [30,25,25,50,120,10,25,30,30,25,35,20,30,40,20,45,25,60,25,30]
available_time_y = [120,120,120,120,120]
available_time_z = [120,120,120,120,120]

# lower(1) and upper(2) bound of nutritions
c_bound=[2700, 3600]
p_bound= [140 ,200]
f_bound=[150 ,250]
s_bound= [2400 ,3000]

# parameter for objective
alpha = 0.01

In [8]:
# Create the model
model = ConcreteModel()
opt = SolverFactory('gurobi_ampl')
model.dual = Suffix(direction=Suffix.IMPORT)

# Define decision Variables
X = []; Y = []; Z = []
for i in range(20):
    X.append('x{0}'.format(i))
    for j in range(5):
        Y.append('y{0}{1}'.format(i,j))
        Z.append('z{0}{1}'.format(i,j))

model.x = Var(X,domain=NonNegativeIntegers)
model.y = Var(Y,domain=NonNegativeIntegers)
model.z = Var(Z,domain=NonNegativeIntegers)

# Define variables for the amount of nutrient, cooking time and expenses
model.fplus = {(i): Var(name='fplus_{0}'.format(i), domain=NonNegativeReals) for i in range(5)}
model.fminus = {(i): Var(name='fminus_{0}'.format(i), domain=NonNegativeReals) for i in range(5)}
model.eplus = Var(name='eplus', domain=NonNegativeReals)
model.eminus = Var(name='eminus', domain=NonNegativeReals)
model.w = Var(name='w', domain=NonNegativeReals)

In [9]:
# look up objective funtion
print(sum(rating[i]*model.x['x{0}'.format(i)] for i in range(20)))

3.81465518848923*x[x0] + 5.0*x[x1] + 4.0*x[x2] + x[x3] + 1.6417068768173502*x[x4] + 3.01206504046907*x[x5] + 2.42544632044816*x[x6] + 4.0*x[x7] + 3.39883700177821*x[x8] + 3.0*x[x9] + 2.18033685383823*x[x10] + 2.4890099337800495*x[x11] + 3.0*x[x12] + 4.0*x[x13] + 3.24024571635759*x[x14] + 3.6957169064405*x[x15] + 2.15972666589039*x[x16] + 2.0*x[x17] + 2.72080467007652*x[x18] + x[x19]


In [10]:
# Define Objective
model.opj = Objective(expr = sum(rating[i]*model.x['x{0}'.format(i)] for i in range(20)) 
             - alpha*model.w, sense=maximize)

In [11]:
#Define constraints
# -- Meal Constraints
model.meals = ConstraintList()
model.meals.add(sum(model.x['x{0}'.format(i)] for i in range(20)) == 5)
model.meals.add(sum(model.y['y{0}{1}'.format(i,j)] for i in range(20) for j in range(5)) <= 5)
model.meals.add(sum(model.z['z{0}{1}'.format(i,j)] for i in range(20) for j in range(5)) <= 5)
for i in range(20):
    model.meals.add(sum(model.y['y{0}{1}'.format(i,j)] + model.z['z{0}{1}'.format(i,j)] for j in range(5)) == model.x['x{0}'.format(i)])
        
# -- Nutrition Bounds
model.nutrition = ConstraintList()
model.nutrition.add(sum(calories[i]*model.x['x{0}'.format(i)] for i in range(20)) >= c_bound[0])
model.nutrition.add(sum(protein[i]*model.x['x{0}'.format(i)] for i in range(20)) >= p_bound[0])
model.nutrition.add(sum(fat[i]*model.x['x{0}'.format(i)] for i in range(20)) >= f_bound[0])
model.nutrition.add(sum(sodium[i]*model.x['x{0}'.format(i)] for i in range(20)) >= s_bound[0])

model.nutrition.add(sum(calories[i]*model.x['x{0}'.format(i)] for i in range(20)) <= c_bound[1])
model.nutrition.add(sum(protein[i]*model.x['x{0}'.format(i)] for i in range(20)) <= p_bound[1])
model.nutrition.add(sum(fat[i]*model.x['x{0}'.format(i)] for i in range(20)) <= f_bound[1])
model.nutrition.add(sum(sodium[i]*model.x['x{0}'.format(i)] for i in range(20)) <= s_bound[1])

# -- Number of recipe
model.number_rec = ConstraintList()
model.number_rec.add(sum(model.x['x{0}'.format(i)] for i in range(20))== 
                       sum(model.y['y{0}{1}'.format(i,j)] for i in range(20) for j in range(5)) + 
                       sum(model.z['z{0}{1}'.format(i,j)] for i in range(20) for j in range(5))
                      )
model.number_rec.add(sum(model.y['y{0}{1}'.format(i,j)] for i in range(20) for j in range(5)) - 
                       sum(model.z['z{0}{1}'.format(i,j)] for i in range(20) for j in range(5)) <= 1 )
model.number_rec.add(-sum(model.y['y{0}{1}'.format(i,j)] for i in range(20) for j in range(5)) + 
                       sum(model.z['z{0}{1}'.format(i,j)] for i in range(20) for j in range(5)) <= 1 )

# -- One recipe a day
model.one_aday = ConstraintList()
for j in range(5):
    model.one_aday.add(sum(model.y['y{0}{1}'.format(i,j)] for i in range(20)) + sum(model.z['z{0}{1}'.format(i,j)] for i in range(20))  
                       == 1)

# -- Time Balance
model.p_time = ConstraintList()
model.p_time.add(sum(p_times[i]* (model.y['y{0}{1}'.format(i,j)] - model.z['z{0}{1}'.format(i,j)]) 
                                        for i in range(20) for j in range(5)) <= model.w)
model.p_time.add(sum(p_times[i]* (-model.y['y{0}{1}'.format(i,j)] + model.z['z{0}{1}'.format(i,j)]) 
                                        for i in range(20) for j in range(5)) <= model.w)
        
# -- No redundant recipe constraint
model.red_rec = ConstraintList()
for j in range(5):
    model.red_rec.add(sum(model.y['y{0}{1}'.format(i,j)] for i in range(20)) <= 1)
    model.red_rec.add(sum(model.z['z{0}{1}'.format(i,j)] for i in range(20)) <= 1)
for i in range(20):    
    model.red_rec.add(sum(model.y['y{0}{1}'.format(i,j)] + model.z['z{0}{1}'.format(i,j)] for j in range(5)) <= 1)
    
# -- Preparation time constraint
model.prepare_time = ConstraintList()
for j in range(5):
    model.prepare_time.add(sum(p_times[i]*model.y['y{0}{1}'.format(i,j)] for i in range(20)) <= available_time_y[j])
    model.prepare_time.add(sum(p_times[i]*model.z['z{0}{1}'.format(i,j)] for i in range(20)) <= available_time_z[j])

In [12]:
# Solve the problem
results = opt.solve(model,tee=True)
results.write()

Academic license - for non-commercial use only
Gurobi 8.1.0# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 81
  Number of variables: 221
  Sense: unknown
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Message: Gurobi 8.1.0\x3a optimal solution; objective 15.5771667353831; 250 simplex iterations; 1 branch-and-cut nodes; plus 1 simplex iteration for intbasis
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.1272139549255371
# ------------------------------------------------------

In [13]:
# Store solution in the model.
model.solutions.store_to(results)
print(results)


Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 81
  Number of variables: 221
  Sense: unknown
Solver: 
- Status: ok
  Message: Gurobi 8.1.0\x3a optimal solution; objective 15.5771667353831; 250 simplex iterations; 1 branch-and-cut nodes; plus 1 simplex iteration for intbasis
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.1272139549255371
Solution: 
- number of solutions: 1
  number of solutions displayed: 1
- Gap: None
  Status: optimal
  Message: Gurobi 8.1.0\x3a optimal solution; objective 15.5771667353831; 250 simplex iterations; 1 branch-and-cut nodes; plus 1 simplex iteration for intbasis
  Objective:
    opj:
      Value: 15.5771667353831
  Variable:
    w:
      Value: 60
    x[x0]:
      Value: 1
    x[x18]:
      Value: 1
    x[x2]:
      Value: 1
    x[x4]:
      Value: 1
    x[x7]:
      Value: 1
    y[y24]:
      Value: 1
    y[y41]:
      Value: 1
    z[z00]:
      Value: 1
    z[z183]:
      Value: 1


In [14]:
def mnmplans(cal_rate=0,pro_rate=0,fat_rate=0,sod_rate=0):
    # Define parameters
    cal_ = 1-(cal_rate/100)
    pro_ = 1+(pro_rate/100)
    fat_ = 1-(fat_rate/100)
    sod_ = 1-(sod_rate/100)
    
    # recipe matrix 
    rating = nutrition['rating'].tolist()
    calories = nutrition['calories'].tolist()
    protein = nutrition['protein'].tolist()
    fat = nutrition['fat'].tolist()
    sodium = nutrition['sodium'].tolist()

    # We have to determine :
    price = [7.2, 5.1,4.3,3.1,4.5,4.3,5.2,9.8,6.1,6.5,4.5,5.5,3.5,13,12.5,4.8,7.5,3.5,6.3,5]
    p_times = [30,25,25,50,120,10,25,30,30,25,35,20,30,40,20,45,25,60,25,30]
    available_time_y = [120,120,120,120,120]
    available_time_z = [120,120,120,120,120]

    # lower(1) and upper(2) bound of nutritions
    c_bound=[2700, 3600]
    p_bound= [140 ,200]
    f_bound=[150 ,250]
    s_bound= [2400 ,3000]
    e_bound= 25

    # parameter for objective
    alpha = 0.1
    
    # Create the model
    model = ConcreteModel()
    opt = SolverFactory('gurobi_ampl')
    model.dual = Suffix(direction=Suffix.IMPORT)

    # Define decision Variables
    X = []; Y = []; Z = []
    for i in range(20):
        X.append('x{0}'.format(i))
        for j in range(5):
            Y.append('y{0}{1}'.format(i,j))
            Z.append('z{0}{1}'.format(i,j))

    model.x = Var(X,domain=NonNegativeIntegers)
    model.y = Var(Y,domain=NonNegativeIntegers)
    model.z = Var(Z,domain=NonNegativeIntegers)

    # Define variables for the amount of nutrient, cooking time and expenses
    model.fplus = {(i): Var(name='fplus_{0}'.format(i), domain=NonNegativeReals) for i in range(5)}
    model.fminus = {(i): Var(name='fminus_{0}'.format(i), domain=NonNegativeReals) for i in range(5)}
    model.eplus = Var(name='eplus', domain=NonNegativeReals)
    model.eminus = Var(name='eminus', domain=NonNegativeReals)
    model.w = Var(name='w', domain=NonNegativeReals)

    # Define Objective
    model.opj = Objective(expr = sum(rating[i]*model.x['x{0}'.format(i)] for i in range(20)) 
             - alpha*model.w, sense=maximize)
    
    #Define constraints
    # -- Meal Constraints
    model.meals = ConstraintList()
    model.meals.add(sum(model.x['x{0}'.format(i)] for i in range(20)) == 5)
    model.meals.add(sum(model.y['y{0}{1}'.format(i,j)] for i in range(20) for j in range(5)) <= 5)
    model.meals.add(sum(model.z['z{0}{1}'.format(i,j)] for i in range(20) for j in range(5)) <= 5)
    for i in range(20):
        model.meals.add(sum(model.y['y{0}{1}'.format(i,j)] + model.z['z{0}{1}'.format(i,j)] for j in range(5)) == model.x['x{0}'.format(i)])

    # -- Nutrition Bounds
    model.nutrition = ConstraintList()
    model.nutrition.add(sum(calories[i]*model.x['x{0}'.format(i)] for i in range(20)) >= c_bound[0])
    model.nutrition.add(sum(protein[i]*model.x['x{0}'.format(i)] for i in range(20)) >= pro_*p_bound[0])
    model.nutrition.add(sum(fat[i]*model.x['x{0}'.format(i)] for i in range(20)) >= f_bound[0])
    model.nutrition.add(sum(sodium[i]*model.x['x{0}'.format(i)] for i in range(20)) >= s_bound[0])

    model.nutrition.add(sum(calories[i]*model.x['x{0}'.format(i)] for i in range(20)) <= cal_*c_bound[1])
    model.nutrition.add(sum(protein[i]*model.x['x{0}'.format(i)] for i in range(20)) <= p_bound[1])
    model.nutrition.add(sum(fat[i]*model.x['x{0}'.format(i)] for i in range(20)) <= fat_*f_bound[1])
    model.nutrition.add(sum(sodium[i]*model.x['x{0}'.format(i)] for i in range(20)) <= sod_*s_bound[1])

    # -- Number of recipe
    model.number_rec = ConstraintList()
    model.number_rec.add(sum(model.x['x{0}'.format(i)] for i in range(20))== 
                           sum(model.y['y{0}{1}'.format(i,j)] for i in range(20) for j in range(5)) + 
                           sum(model.z['z{0}{1}'.format(i,j)] for i in range(20) for j in range(5))
                          )
    model.number_rec.add(sum(model.y['y{0}{1}'.format(i,j)] for i in range(20) for j in range(5)) - 
                       sum(model.z['z{0}{1}'.format(i,j)] for i in range(20) for j in range(5)) <= 1 )
    model.number_rec.add(-sum(model.y['y{0}{1}'.format(i,j)] for i in range(20) for j in range(5)) + 
                       sum(model.z['z{0}{1}'.format(i,j)] for i in range(20) for j in range(5)) <= 1 )

    # -- One recipe a day
    model.one_aday = ConstraintList()
    for j in range(5):
        model.one_aday.add(sum(model.y['y{0}{1}'.format(i,j)] for i in range(20)) + sum(model.z['z{0}{1}'.format(i,j)] for i in range(20))  
                           == 1)
    
    # -- Time Balance
    model.p_time = ConstraintList()
    model.p_time.add(sum(p_times[i]* (model.y['y{0}{1}'.format(i,j)] - model.z['z{0}{1}'.format(i,j)]) 
                                            for i in range(20) for j in range(5)) <= model.w)
    model.p_time.add(sum(p_times[i]* (-model.y['y{0}{1}'.format(i,j)] + model.z['z{0}{1}'.format(i,j)]) 
                                            for i in range(20) for j in range(5)) <= model.w)

    # -- No redundant recipe constraint
    model.red_rec = ConstraintList()
    for j in range(5):
        model.red_rec.add(sum(model.y['y{0}{1}'.format(i,j)] for i in range(20)) <= 1)
        model.red_rec.add(sum(model.z['z{0}{1}'.format(i,j)] for i in range(20)) <= 1)
    for i in range(20):    
        model.red_rec.add(sum(model.y['y{0}{1}'.format(i,j)] + model.z['z{0}{1}'.format(i,j)] for j in range(5)) <= 1)

    # -- Preparation time constraint
    model.prepare_time = ConstraintList()
    for j in range(5):
        model.prepare_time.add(sum(p_times[i]*model.y['y{0}{1}'.format(i,j)] for i in range(20)) <= available_time_y[j])
        model.prepare_time.add(sum(p_times[i]*model.z['z{0}{1}'.format(i,j)] for i in range(20)) <= available_time_z[j])
        
    results = opt.solve(model,tee=True)
    model.solutions.store_to(results)
    return results

mnmplans().write()

Academic license - for non-commercial use only
Gurobi 8.1.0# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 81
  Number of variables: 221
  Sense: unknown
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Message: Gurobi 8.1.0\x3a optimal solution; objective 10.1771667353831; 298 simplex iterations; 1 branch-and-cut nodes; plus 1 simplex iteration for intbasis
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.08051395416259766
# -----------------------------------------------------

In [15]:
mnmplans(cal_rate=20).write()

Academic license - for non-commercial use only
Gurobi 8.1.0# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 81
  Number of variables: 221
  Sense: unknown
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Message: Gurobi 8.1.0\x3a optimal solution; objective 10.1771667353831; 366 simplex iterations; 1 branch-and-cut nodes; plus 1 simplex iteration for intbasis
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.10976910591125488
# -----------------------------------------------------